In [4]:
! pip install azure-search-documents langchain langchain-openai

     -------------------------------------- 297.7/297.7 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 21.2 MB/s eta 0:00:00
     ---------------------------------------- 49.9/49.9 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 162.0/162.0 kB ? eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 26.7 MB/s eta 0:00:00
     ------------------------------------- 381.6/381.6 kB 23.2 MB/s eta 0:00:00
     ------------------------------------- 407.7/407.7 kB 24.8 MB/s eta 0:00:00
     ---------------------------------------- 296.9/296.9 kB ? eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 34.4 MB/s eta 0:00:00
     ------------------------------------- 884.5/884.5 kB 28.2 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ---------------------------------------- 63.0/63.0 kB ? eta 0:00:00
     ---------------------------------------- 50.5/50.5 kB 2.7 MB/s eta 0:00:00
  


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from openai import OpenAI
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

In [6]:
kv_uri = f"https://kv-poligpt-dev-eastus2.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

azure_search_endpoint = client.get_secret("azure-search-endpoint").value
azure_search_admin_key = client.get_secret("azure-search-admin-key").value
azure_search_credential = AzureKeyCredential(azure_search_admin_key)

openai_api_key = client.get_secret("openai-api-key").value
embedding_model = "text-embedding-3-large"
embedding_dimensions = 3072
text_generation_model = "gpt-4"
index_name = "poligpt-index"

In [7]:

# Função que gera embeddings a partir de texto usando a API da OpenAI
def get_embeddings(text, model, dimensions):
    client = OpenAI(api_key = openai_api_key)
    text = text.replace("\n", " ")

    response = client.embeddings.create(
        input=[text],
        model=model,
        dimensions=dimensions
    )

    return response.data[0].embedding

In [13]:
# Hybrid Search
query = "Como proceder para registrar estágio obrigatório?"  
query_embedding = get_embeddings(text=query, model=embedding_model, dimensions=embedding_dimensions)

search_client = SearchClient(endpoint=azure_search_endpoint, index_name=index_name, credential=azure_search_credential)
vector_query = VectorizedQuery(vector=query_embedding, k_nearest_neighbors=10, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["chunk_id", "title", "keywords", "last_modified", "chunk"],
    top=5
)  
  
for result in results:  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"title: {result['title']}")
    print(f"keywords: {result['keywords']}")  
    print(f"last_modified: {result['last_modified']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  

chunk_id: 90a4571bb413_e50bcc3e19138efe24a05d9722de0694_pages_4
title: Programa de Estágio - Politécnica UFRJ
keywords: estudante,programa de estagios
last_modified: 06/24/2024 00:00:00 +00:00
Score: 0.032522473484277725
Content: de Estágio Obrigatório
ou o(s)
Registro(s) de Estágio Não Obrigatório
, conforme modelos abaixo:
Para que as horas referentes ao estágio sejam lançadas no boletim, o(a) aluno(a) deve enviar o relatório de estágio e o registro de estágio obrigatório ou não obrigatório (já assinados pela comissão de estágio do curso e coordenação do curso) pelo formulário “
Procedimentos de ACE
” (necessário estar logado com o polimail) pelo
Atendimento Virtual
.
Para lançamento de outros tipos de ACE que não sejam de estágio, o envio deve ser feito diretamente à Secretaria Acadêmica do curso.
Todos Os Formulários e Modelos de Documentos
Análise econômica da DAPE para realização de Estágio:
Caso o(a) aluno(a) não possua os requisitos para fazer o estágio, é possível fazer uma an

---

In [14]:
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.llm import LLMChain
from langchain.schema import AIMessage, HumanMessage

In [15]:
# Configuração do GPT-4 no Langchain
llm = ChatOpenAI(model=text_generation_model, temperature=0, api_key=openai_api_key)

# Definir um prompt para GPT-4
template = """Usando os seguintes documentos como contexto, responda à pergunta 
(em caso de achar suficiente, diga que não sabe):

{query}

Documentos:
{context}
"""

prompt = PromptTemplate(
    input_variables=["query", "context"],
    template=template
)

# Função que envia o contexto (documentos) para o GPT-4 e gera a resposta
def generate_answer(query, documents):
    context = "\n\n".join([doc for doc in documents])
    
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(query=query, context=context)
    
    return response

response = generate_answer(query, results)
print(response)

Os documentos não foram fornecidos, portanto, não é possível fornecer uma resposta precisa sobre como proceder para registrar um estágio obrigatório.


In [18]:
prompt

PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='Usando os seguintes documentos como contexto, responda à pergunta \n(em caso de achar suficiente, diga que não sabe):\n\n{query}\n\nDocumentos:\n{context}\n')